In [1]:
import torch
import torch.nn as nn

In [2]:
# Load the data
import pandas as pd

train_path = '../data/train.csv'
dev_in_path = '../data/dev_in.csv'

df_train = pd.read_csv(train_path)
df_train.head()

,fact_time,fact_latitude,fact_longitude,fact_temperature,fact_cwsm_class,climate,topography_bathymetry,sun_elevation,climate_temperature,climate_pressure,...,cmc_0_1_66_0_grad,cmc_0_1_66_0_next,cmc_0_1_67_0_grad,cmc_0_1_67_0_next,cmc_0_1_68_0_grad,cmc_0_1_68_0_next,gfs_2m_dewpoint_grad,gfs_2m_dewpoint_next,gfs_total_clouds_cover_low_grad,gfs_total_clouds_cover_low_next
0,1.543321e+09,26.968800,-99.248901,2.0,0.0,dry,127.0,-17.526443,14.613571,754.263405,...,0.0,0.0,0.0,0.0,0.0,0.0,-2.600006,-2.750006,0.0,0.0
1,1.538776e+09,29.374201,-100.927002,31.0,20.0,mild temperate,297.0,41.531032,26.992143,733.117168,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.600006,17.950006,-12.0,11.0
2,1.552115e+09,22.149599,113.592003,17.0,10.0,mild temperate,-1.0,43.916531,18.842143,761.571076,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.233978,21.450006,1.0,8.0
3,1.549566e+09,34.678699,-86.684799,24.0,20.0,mild temperate,193.0,40.240955,8.303571,747.524910,...,0.0,0.0,0.0,0.0,0.0,0.0,0.059448,16.150018,-58.0,41.0
4,1.552910e+09,46.066667,41.966667,9.0,20.0,dry,90.0,30.394660,6.451429,753.168113,...,0.0,0.0,0.0,0.0,0.0,0.0,0.400024,3.150018,18.0,92.0


In [3]:
df_dev_in = pd.read_csv(dev_in_path)
df_dev_in.head()

,fact_time,fact_latitude,fact_longitude,fact_temperature,fact_cwsm_class,climate,topography_bathymetry,sun_elevation,climate_temperature,climate_pressure,...,cmc_0_1_66_0_grad,cmc_0_1_66_0_next,cmc_0_1_67_0_grad,cmc_0_1_67_0_next,cmc_0_1_68_0_grad,cmc_0_1_68_0_next,gfs_2m_dewpoint_grad,gfs_2m_dewpoint_next,gfs_total_clouds_cover_low_grad,gfs_total_clouds_cover_low_next
0,1539162000,-40.350000,-9.880000,11.0,10,tropical,-843.0,31.782490,10.070714,765.631228,...,0.00000,0.0000,0.0,0.000000,0.0,0.0,0.505035,2.647577,2.0,2.0
1,1545006600,53.421299,-6.270070,4.0,10,mild temperate,67.0,-59.691521,7.005000,752.897615,...,0.00000,0.0000,0.0,0.000000,0.0,0.0,-0.400024,1.249994,0.0,0.0
2,1540094400,-19.757700,63.361000,26.0,10,dry,6.0,35.250889,23.327143,763.115016,...,0.00000,0.0000,0.0,0.000000,0.0,0.0,0.100006,21.050012,-1.0,1.0
3,1552611600,35.245899,47.009201,5.0,10,mild temperate,1390.0,-23.755615,3.109286,609.419333,...,1.69672,5.1653,0.0,0.000049,0.0,0.0,-1.500000,-0.349982,-12.0,81.0
4,1545631200,26.633333,118.150000,14.0,20,mild temperate,210.0,33.040438,12.172143,734.678037,...,0.00000,0.0000,0.0,0.000000,0.0,0.0,0.102081,11.513879,-15.0,83.0


In [4]:
# Identify the categorical features
cat_features = []
for col in df_dev_in:
    values = df_dev_in[col].tolist()
    unique = list(dict.fromkeys(values))
    if len(unique) < 20:
        cat_features.append(col)
print(cat_features)

['fact_cwsm_class', 'climate', 'cmc_available', 'gfs_available', 'gfs_soil_temperature_available', 'gfs_timedelta_s', 'wrf_available']


In [5]:
from scipy import stats
import numpy as np

nan_replacements = {}
for col in df_train:
    if col in cat_features:
        nan_replacements[col] = stats.mode(np.asarray(df_train[col].tolist()))[0][0]
        # print(nan_replacements[col])
    else:
        nan_replacements[col] = np.mean(np.asarray(df_train[col].dropna().tolist()))
print(nan_replacements.values())

dict_values([1545219635.2602534, 27.3491997995972, -19.454721235367906, 15.030736293991083, 0.0, 'mild temperate', 309.93658796685, -2.7249874731371273, 15.053651434869764, 731.2756530851661, 289.1659257771938, 0.0513848381221949, 287.04881832634976, 287.0853175734476, 287.0339327353252, 258.9922501121575, 274.97475431188, 282.77351197349384, 285.8909051108904, 281.3276019215079, 7.660611531165116, 5.534873822269684, 14.704511863943937, 12.971739936778564, 8.78866233485156, 7.2775056043593676, 0.009529923266778687, 0.00859610917297107, 6.413124162323838, 0.2673820249969054, 0.027512247376225554, 0.008222988603064724, 3.866966535018241e-05, -0.0020003342605871715, -0.05948973513452053, 10.42531664950694, 5.2929354486362215, 1.8376276146288066, 0.4768480677913728, -0.09701087752828826, -0.14489588205685547, 0.22814633531193007, 0.2219977889627698, 0.30237136633108197, 0.18710502254215602, 97345.58898593165, 97344.2322319218, 101661.80541725569, 137.95607755299883, 5728.521789532874, 3094

In [6]:
# Replace nans in train and dev
for col in df_train:
    df_train[col] = df_train[col].fillna(nan_replacements[col])
    df_dev_in[col] = df_dev_in[col].fillna(nan_replacements[col])

In [7]:
from typing import Dict

import numpy as np
import sklearn.preprocessing


def normalize(
    X: Dict[str, np.ndarray], normalization: str, seed: int, noise: float = 1e-3
) -> Dict[str, np.ndarray]:
    # X ~ {'train': <train_size x n_features>, 'val': <val_size x n_features>, 'test': <test_size x n_features>}
    X_train = X['train']
    if normalization == 'standard':
        normalizer = sklearn.preprocessing.StandardScaler()
    elif normalization == 'quantile':
        normalizer = sklearn.preprocessing.QuantileTransformer(
            output_distribution='normal',
            n_quantiles=max(min(X['train'].shape[0] // 30, 1000), 10),
            subsample=1e9,
            random_state=seed,
        )
        if noise:
            X_train = X_train.copy()
            stds = np.std(X_train, axis=0, keepdims=True)
            noise_std = noise / np.maximum(stds, noise)
            X_train += noise_std * np.random.default_rng(seed).standard_normal(
                X_train.shape
            )
    else:
        raise ValueError(f'unknown normalization: {normalization}')
    normalizer.fit(X_train)
    return {k: normalizer.transform(v) for k, v in X.items()}

In [8]:
# Set Seed
seed = 1
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

In [9]:
# Normalise using train data stats
# Quantile normalisation is used (maps to a normal distribution)
X_train_np = np.asarray(df_train.iloc[:,6:])
X_dev_in_np = np.asarray(df_dev_in.iloc[:,6:])
X = {'train': X_train_np, 'dev_in': X_dev_in_np}
X = normalize(X, normalization='quantile', seed=seed)
X_train_np = X['train']
X_dev_in_np = X['dev_in']

X_train = torch.FloatTensor(X_train_np)
X_dev_in = torch.FloatTensor(X_dev_in_np)

In [10]:
# Preprocess into tensors

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

def get_lab_to_ind(data_df):
    '''
    Prepare a label to index map
    '''
    y_fact = set(list(data_df['fact_cwsm_class']))
    lab_to_ind = {}
    for i, lab in enumerate(y_fact):
        lab_to_ind[lab] = i
    return lab_to_ind

lab_to_ind = get_lab_to_ind(df_train)
batch_size = 1024

# Train
y_train = np.asarray(df_train['fact_cwsm_class'])
y_train = torch.LongTensor(np.asarray([lab_to_ind[lab] for lab in y_train]))

train_ds = TensorDataset(X_train, y_train)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

# Dev in
y_dev_in = df_dev_in['fact_cwsm_class']
y_dev_in = torch.LongTensor(np.asarray([lab_to_ind[lab] for lab in y_dev_in]))

dev_in_ds = TensorDataset(X_dev_in, y_dev_in)
dev_in_dl = DataLoader(dev_in_ds, batch_size=batch_size, shuffle=True)

In [11]:
# Get the device

def get_default_device():
    if torch.cuda.is_available():
        print("Got CUDA!")
        return torch.device('cuda')
    else:
        print("No CUDA found")
        return torch.device('cpu')

device = get_default_device()

No CUDA found


In [31]:
# Define MLP Model

class MLP(nn.Module):
    '''
        Multi-Layer Perceptron
    '''
    def __init__(self, num_feats=123, num_classes=9):
        super().__init__()
        
        layer_size = 256
        dropout_p = 0.25

        self.ffn = nn.Sequential(
            nn.Linear(num_feats, layer_size),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            nn.Linear(layer_size, num_classes),
        )
    
    def forward(self, x):
        return self.ffn(x)

model = MLP()
model.to(device)

MLP(
  (ffn): Sequential(
    (0): Linear(in_features=123, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
    (3): Linear(in_features=256, out_features=256, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.25, inplace=False)
    (6): Linear(in_features=256, out_features=256, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.25, inplace=False)
    (9): Linear(in_features=256, out_features=256, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.25, inplace=False)
    (12): Linear(in_features=256, out_features=256, bias=True)
    (13): ReLU()
    (14): Dropout(p=0.25, inplace=False)
    (15): Linear(in_features=256, out_features=256, bias=True)
    (16): ReLU()
    (17): Dropout(p=0.25, inplace=False)
    (18): Linear(in_features=256, out_features=256, bias=True)
    (19): ReLU()
    (20): Dropout(p=0.25, inplace=False)
    (21): Linear(in_features=256, out_features=9, bias=True)
  )
)

In [32]:
# optimizer

lr = 0.005
weight_decay = 0.9

optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

In [33]:
# Criterion
criterion = nn.CrossEntropyLoss().to(device)

In [34]:
# Training and Evaluation Function

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def accuracy_topk(output, target, k=1):
    """Computes the topk accuracy"""
    batch_size = target.size(0)

    _, pred = torch.topk(output, k=k, dim=1, largest=True, sorted=True)

    res_total = 0
    for curr_k in range(k):
      curr_ind = pred[:,curr_k]
      num_eq = torch.eq(curr_ind, target).sum()
      acc = num_eq/len(output)
      res_total += acc
    return res_total*100

def train(train_loader, model, criterion, optimizer, epoch, device, print_freq=100):
    '''
    Run one train epoch
    '''
    losses = AverageMeter()
    accs = AverageMeter()

    # switch to train mode
    model.train()

    for i, (x, target) in enumerate(train_loader):

        x = x.to(device)
        target = target.to(device)

        # Forward pass
        logits = model(x)
        loss = criterion(logits, target)

        # Backward pass and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure accuracy and record loss
        acc = accuracy_topk(logits.data, target)
        accs.update(acc.item(), x.size(0))
        losses.update(loss.item(), x.size(0))

        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                    'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                    'Accuracy {prec.val:.3f} ({prec.avg:.3f})'.format(
                      epoch, i, len(train_loader),
                      loss=losses, prec=accs))

@torch.no_grad()
def eval(val_loader, model, criterion, device):
    '''
    Run evaluation
    '''
    losses = AverageMeter()
    accs = AverageMeter()

    # switch to eval mode
    model.eval()


    for i, (x, target) in enumerate(val_loader):

        x = x.to(device)
        target = target.to(device)

        # Forward pass
        logits = model(x)
        loss = criterion(logits, target)

        # measure accuracy and record loss
        acc = accuracy_topk(logits.data, target)
        accs.update(acc.item(), x.size(0))
        losses.update(loss.item(), x.size(0))

    print('Dev in\t Loss ({loss.avg:.4f})\t'
            'Accuracy ({prec.avg:.3f})\n'.format(
              loss=losses, prec=accs))

In [35]:
# Train
epochs = 10
for epoch in range(epochs):

    # train for one epoch
    print('current lr {:.5e}'.format(optimizer.param_groups[0]['lr']))
    train(train_dl, model, criterion, optimizer, epoch, device)

    # evaluate on validation set
    eval(dev_in_dl, model, criterion, device)

current lr 5.00000e-03
Epoch: [0][0/3057]	Loss 2.2115 (2.2115)	Accuracy 3.613 (3.613)
Epoch: [0][100/3057]	Loss 1.1086 (1.1954)	Accuracy 51.562 (46.861)
Epoch: [0][200/3057]	Loss 1.0769 (1.1384)	Accuracy 54.688 (50.642)
Epoch: [0][300/3057]	Loss 1.1075 (1.1149)	Accuracy 53.613 (52.281)
Epoch: [0][400/3057]	Loss 1.0981 (1.0998)	Accuracy 57.227 (53.373)
Epoch: [0][500/3057]	Loss 1.0309 (1.0888)	Accuracy 58.105 (54.090)
Epoch: [0][600/3057]	Loss 1.0161 (1.0807)	Accuracy 58.398 (54.643)
Epoch: [0][700/3057]	Loss 1.0414 (1.0751)	Accuracy 57.227 (55.009)
Epoch: [0][800/3057]	Loss 1.0335 (1.0701)	Accuracy 57.812 (55.323)
Epoch: [0][900/3057]	Loss 1.0386 (1.0666)	Accuracy 56.934 (55.530)
Epoch: [0][1000/3057]	Loss 1.0857 (1.0636)	Accuracy 55.957 (55.711)
Epoch: [0][1100/3057]	Loss 1.0010 (1.0611)	Accuracy 60.059 (55.865)
Epoch: [0][1200/3057]	Loss 0.9837 (1.0587)	Accuracy 58.594 (56.025)
Epoch: [0][1300/3057]	Loss 1.0508 (1.0570)	Accuracy 56.152 (56.127)
Epoch: [0][1400/3057]	Loss 1.0820 (1.05

KeyboardInterrupt: 